In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID, EGA out, RT.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    1.555000e+03    1.555000e+03     1.555000e+03     1.555000e+03   
mean     5.265489e-11    2.398615e-12     9.698748e-13     2.423962e-12   
std      1.606675e-11    6.591833e-13     4.188290e-13     1.305344e-12   
min      2.033520e-11    9.424000e-13     2.322000e-13     4.578000e-13   
25%      4.216280e-11    2.039650e-12     5.657500e-13     1.032900e-12   
50%      5.850630e-11    2.685400e-12     1.136400e-12     2.792400e-12   
75%      6.227930e-11    2.807000e-12     1.225650e-12     3.267950e-12   
max      9.602170e-11    3.635200e-12     1.823700e-12     1.067000e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     1.555000e+03     1.555000e+03     1.555000e+03     1.342000e+03   
mean      1.687716e-12     5.714817e-12     1.402769e-13     5.932120e-12   
std       2.945910e-13     2.065153e-12     3.181548e-13     2.781177e-12   
min       1.0324

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/14 15:34:40.794,2022/11/14 15:34:41.044,2022/11/14 15:34:41.294,2022/11/14 15:34:41.544,2022/11/14 15:34:41.794,2022/11/14 15:34:42.044,2022/11/14 15:34:42.295,2022/11/14 15:34:42.545,2022/11/14 15:34:42.795,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2022/11/14 15:34:43.117,2022/11/14 15:34:43.367,2022/11/14 15:34:43.617,2022/11/14 15:34:43.868,2022/11/14 15:34:44.118,2022/11/14 15:34:44.368,2022/11/14 15:34:44.618,2022/11/14 15:34:44.868,2022/11/14 15:34:45.119,2.323,2.323,2.323,2.324,2.324,2.324,2.323,2.323,2.324
4,2022/11/14 15:34:45.440,2022/11/14 15:34:45.690,2022/11/14 15:34:45.941,2022/11/14 15:34:46.191,2022/11/14 15:34:46.441,2022/11/14 15:34:46.691,2022/11/14 15:34:46.941,2022/11/14 15:34:47.192,2022/11/14 15:34:47.442,4.646,4.646,4.647,4.647,4.647,4.647,4.646,4.647,4.647
5,2022/11/14 15:34:47.763,2022/11/14 15:34:48.013,2022/11/14 15:34:48.263,2022/11/14 15:34:48.513,2022/11/14 15:34:48.763,2022/11/14 15:34:49.014,2022/11/14 15:34:49.264,2022/11/14 15:34:49.764,NaN,6.969,6.969,6.969,6.969,6.969,6.970,6.969,7.219,NaN
6,2022/11/14 15:34:50.085,2022/11/14 15:34:50.335,2022/11/14 15:34:50.586,2022/11/14 15:34:50.836,2022/11/14 15:34:51.086,2022/11/14 15:34:51.336,2022/11/14 15:34:51.586,2022/11/14 15:34:52.087,NaN,9.291,9.291,9.292,9.292,9.292,9.292,9.291,9.542,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,2022/11/14 16:34:40.670,2022/11/14 16:34:40.920,2022/11/14 16:34:41.171,2022/11/14 16:34:41.421,2022/11/14 16:34:41.671,2022/11/14 16:34:41.921,2022/11/14 16:34:42.171,2022/11/14 16:34:42.672,NaN,3599.876,3599.876,3599.877,3599.877,3599.877,3599.877,3599.876,3600.127,NaN
1553,2022/11/14 16:34:42.993,2022/11/14 16:34:43.243,2022/11/14 16:34:43.493,2022/11/14 16:34:43.743,2022/11/14 16:34:43.993,2022/11/14 16:34:44.244,2022/11/14 16:34:44.494,2022/11/14 16:34:44.994,NaN,3602.199,3602.199,3602.199,3602.199,3602.199,3602.200,3602.199,3602.449,NaN
1554,2022/11/14 16:34:45.315,2022/11/14 16:34:45.565,2022/11/14 16:34:45.816,2022/11/14 16:34:46.066,2022/11/14 16:34:46.316,2022/11/14 16:34:46.566,2022/11/14 16:34:46.816,2022/11/14 16:34:47.317,NaN,3604.521,3604.521,3604.522,3604.522,3604.522,3604.522,3604.521,3604.772,NaN
1555,2022/11/14 16:34:47.638,2022/11/14 16:34:47.888,2022/11/14 16:34:48.138,2022/11/14 16:34:48.388,2022/11/14 16:34:48.638,2022/11/14 16:34:48.889,2022/11/14 16:34:49.139,2022/11/14 16:34:49.639,NaN,3606.844,3606.844,3606.844,3606.844,3606.844,3606.845,3606.844,3607.094,NaN


### Position of the EGA

 

<div>
<img src="20221114_144204.jpg" width="1000"/>
</div>

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,14/11/2022 15:32:53,10.0,-0.063995,2.120000e-09,8.100000e-10,4.996000e-09,7.856000e-07,0.000272,0.0,-98.540,62.514,0.0,NaN
2,NaN,14/11/2022 15:33:02,19.0,-0.067398,2.070000e-09,7.920000e-10,4.996000e-09,7.856000e-07,0.000240,0.0,-98.428,62.514,0.0,NaN
3,NaN,14/11/2022 15:33:11,28.0,-0.067496,2.050000e-09,7.890000e-10,4.996000e-09,7.856000e-07,0.000266,0.0,-98.540,62.514,0.0,NaN
4,NaN,14/11/2022 15:33:20,37.0,-0.062665,2.030000e-09,7.890000e-10,4.996000e-09,7.856000e-07,0.000249,0.0,-98.540,62.401,0.0,NaN
5,NaN,14/11/2022 15:33:29,46.0,-0.066832,2.020000e-09,7.820000e-10,4.996000e-09,7.856000e-07,0.000274,0.0,-98.540,62.626,0.0,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3201.0,3201.0,3.201000e+03,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3201.000000,3.201000e+03,3.201000e+03
mean,1919.075620,104.134146,81.560975,815.711346,816.800137,864.195208,0.0,0.0,4.000000e-02,-0.142011,0.447251,27.756642,27.876371,29.488804,-0.009891,-0.014429,-0.011978,1.784143e-07,5.363467e-07
std,1110.414843,202.377257,38.841256,386.778081,385.997930,342.296526,0.0,0.0,2.748231e-15,0.164804,0.507946,16.821632,16.542738,12.778766,0.001722,0.000793,0.000894,1.244676e-07,4.127318e-07
min,1.172000,0.060000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.000000e-02,-0.564800,-3.332900,-24.764300,-22.811800,0.010300,-0.013700,-0.016600,-0.014000,2.500000e-09,6.500000e-09
25%,957.395000,0.120000,100.060000,1000.000000,1000.080000,1000.120000,0.0,0.0,4.000000e-02,-0.266800,0.016200,34.282400,34.309000,34.297300,-0.011300,-0.014800,-0.012600,1.710000e-08,1.700000e-08
50%,1918.918000,0.140000,100.060000,1000.020000,1000.120000,1000.140000,0.0,0.0,4.000000e-02,-0.229600,0.727100,35.544700,35.577900,35.562000,-0.009800,-0.014300,-0.012100,2.365000e-07,7.494000e-07
75%,2879.973000,0.160000,100.080000,1000.040000,1000.140000,1000.140000,0.0,0.0,4.000000e-02,0.015100,0.774200,35.891000,35.926800,35.910500,-0.008700,-0.013800,-0.011200,2.684000e-07,7.796000e-07
max,3844.776000,499.940000,100.120000,1000.040000,1000.160000,1000.480000,0.0,0.0,4.000000e-02,0.528200,3.676700,36.167200,36.203500,36.185100,-0.006900,-0.013500,-0.009700,5.648000e-07,3.676700e-06


### Data processing

In [5]:
#Filtering background signal

#Find the timestamp where emission was turned on
print("Result 1:",hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 11, 14, 17,12, 45)
#y=pd.Timestamp(2022, 11, 10, 17,12, 45)
#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#average the signal before this timestamp and subtract it from the Em-on part
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 1:",mid["mass 2, corr"].describe())
#remove signal below 10^-14
mid["mass 32.0_value"][mid["mass 32.0_value"] < 1e-14] = np.nan

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][102:195]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:102]/h2_sens
base_h2.describe()

Result 1:             Date          Time  Elapsed time   VMon1   VMon2    VMon3  \
1182  14/11/2022  15:57:38.930      1415.726  200.80  100.08  1000.04   
1183  14/11/2022  15:57:40.132      1416.928  291.24  100.10  1000.04   
1184  14/11/2022  15:57:41.330      1418.126  421.34  100.10  1000.04   
1185  14/11/2022  15:57:42.618      1419.414  499.36  100.10  1000.04   

        VMon4    VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5  \
1182  1000.14  1000.14    0.0    0.0  ...  0.2141  35.4478  35.4276  35.4056   
1183  1000.14  1000.14    0.0    0.0  ...  0.0798  35.4216  35.4258  35.4056   
1184  1000.14  1000.14    0.0    0.0  ...  0.0550  35.4025  35.4214  35.4041   
1185  1000.14  1000.14    0.0    0.0  ...  0.0399  35.3948  35.4204  35.4027   

       IMon6   IMon7   IMon8         Comment       hv_grid            em  
1182 -0.0103 -0.0140 -0.0109  grid repelling  4.611000e-07  2.141000e-07  
1183 -0.0103 -0.0138 -0.0109  grid repelling  4.915000e-07  7.980000e-08 

C:\Users\etiirine\AppData\Local\Temp\ipykernel_3484\765137446.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 32.0_value"][mid["mass 32.0_value"] < 1e-14] = np.nan


count    1.020000e+02
mean     6.913702e-10
std      5.603373e-12
min      6.762009e-10
25%      6.882890e-10
50%      6.915319e-10
75%      6.949310e-10
max      7.119099e-10
Name: mass 2, corr, dtype: float64

In [6]:
#CO

#Find the timestamp where emission was turned on
#print("Result 1:",hv.loc[hv["Comment"]=="emission start"])
x=pd.Timestamp(2022, 11, 10, 17,12, 45)

#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 28.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#average this signal and subtract it from the Em-on part
bck=np.mean(mid["mass 28.0_value"][0:102])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 28, corr"]=mid["mass 28.0_value"][102:]-bck
mid["mass 28, corr"].describe()

#getting the differential spectrum
#first get the timestamp where grid voltage was changed to -500V
print("Location:",hv.loc[hv["Comment"]=="grid repelling"])
y=pd.Timestamp(2022, 11, 10, 17,22, 56)

#then find the time value closest to it in the MID scan file
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 28.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#subtract grid repelling from grid transparent
#getting the mean signal when grid is repelling
grid2=np.mean(mid["mass 28, corr"][195:])
mid["sample sig, mass 28"]=mid["mass 28, corr"][102:195]-grid2
mid["sample sig, mass 28"].describe()



 Result 2:                 mass 2.0_time  mass 2.0_value            mass 4.0_time  \
2     2022/11/14 15:34:40.794    2.109580e-11  2022/11/14 15:34:41.044   
3     2022/11/14 15:34:43.117    2.059160e-11  2022/11/14 15:34:43.367   
4     2022/11/14 15:34:45.440    2.072110e-11  2022/11/14 15:34:45.690   
5     2022/11/14 15:34:47.763    2.066310e-11  2022/11/14 15:34:48.013   
6     2022/11/14 15:34:50.085    2.067110e-11  2022/11/14 15:34:50.335   
...                       ...             ...                      ...   
1552  2022/11/14 16:34:40.670    6.015750e-11  2022/11/14 16:34:40.920   
1553  2022/11/14 16:34:42.993    6.047300e-11  2022/11/14 16:34:43.243   
1554  2022/11/14 16:34:45.315    6.017800e-11  2022/11/14 16:34:45.565   
1555  2022/11/14 16:34:47.638    6.030480e-11  2022/11/14 16:34:47.888   
1556  2022/11/14 16:34:49.960    6.011450e-11  2022/11/14 16:34:50.210   

      mass 4.0_value           mass 12.0_time  mass 12.0_value  \
2       1.032500e-12  2022/11/14 

count    9.300000e+01
mean    -4.698176e-12
std      3.081149e-14
min     -4.776384e-12
25%     -4.714084e-12
50%     -4.696484e-12
75%     -4.681284e-12
max     -4.633784e-12
Name: sample sig, mass 28, dtype: float64

### Plotting

In [7]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan, EGA out @RT, dynamic vacuum")
plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("HiVolta current recording, EGA out @RT, dynamic vacuum")
plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"]/3600,vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution, EGA out @RT, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [10]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 296
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
I_em = np.mean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


7.407922126900058e-10
0.01992516581153798
1.9949766666666664e-06


In [11]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.00028980756450685375 

